In [1]:
from dataHelpers import proteinNames, importLINCSprotein

In [37]:
import numpy as np
import pandas as pd
#from tensorly.decomposition import  partial_tucker
from Data_Mod import form_tensor
from tensor import partial_tucker_decomp, find_R2X_partialtucker, flip_factors

def get_Flattened_Matrices(result, data, treatment_list, times):
    #create a 5x42 DataFrame of decompsed component values
    toflatten = result[0] #treatment/comp
    flattened = []

    for treatment in range(7):
        flattened.append(pd.DataFrame(toflatten[treatment]).T)

    df = pd.concat(flattened, axis=1)
    
    #component values for control at T=0
    ctrl = df.iloc[:, 0]

    #Remove all times that aren't 24 or 48
    #df.drop([0,1,2,3], inplace = True, axis = 1)
    
    #add control back in
    ctrl = pd.DataFrame(ctrl)
    df = pd.concat([ctrl,df], axis = 1)

    #rename columns
    cols = [str(x) + "_" + str(y) for x in treatment_list for y in times[-2:]]
    cols = ["ctrl_0"] + cols
    df.columns = cols

    #sort by column name
    df = df.reindex(sorted(df.columns), axis=1)
    data = data.reindex(sorted(data.columns), axis=1)

    #make dataframes into numpy arrays
    df = df.to_numpy()
    data = data.to_numpy()
    return df, data

def get_reconstruct(P,X):
    Ppinv = np.linalg.pinv(P.T)
    return Ppinv, np.matmul(Ppinv,X.T)

def find_gene_factors(result, data, treatment_list, times):
    P, X = get_Flattened_Matrices(result, data, treatment_list, times)
    Ppinv, W = get_reconstruct(P, X)
    return P, X, Ppinv, W

In [42]:
component = 10
tensor, treatment_list, times, df = form_tensor()
pre_flip_result = partial_tucker_decomp(tensor, [2], component)

result = flip_factors(pre_flip_result)

In [41]:
protein = importLINCSprotein()
protein = protein.drop(["Sample description"], axis=1)
protein = protein.drop([0])
by_row_index = df.groupby(df.index)
protein = protein.drop(columns=['Treatment', 'Time', 'File'])

In [38]:
P, X = get_Flattened_Matrices(result, protein, treatment_list, times)
print(P, X)

ValueError: Length mismatch: Expected axis has 43 elements, new values have 15 elements

In [ ]:
Gene_redone = np.matmul(W.T, P)

In [ ]:
1 - tl_var(Gene_redone - X) / tl_var(RNAseq.to_numpy())